In [15]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import pytesseract
import io
import base64
import re
import pandas as pd

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

# Função para raspar as imagens do site
def scrape_images(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    images = [img['src'] for img in soup.find_all('img', class_='img-thumbnail') if img.get('src')]
    return images

# Função para extrair o texto de imagens (HTTP ou base64)
def ocr_image(image_data):
    if image_data.startswith('http'):
        response = requests.get(image_data)
        image = Image.open(io.BytesIO(response.content))
    elif image_data.startswith('data:image'):
        base64_data = image_data.split(',', 1)[1]
        image_bytes = base64.b64decode(base64_data)
        image = Image.open(io.BytesIO(image_bytes))
    else:
        raise ValueError("Unsupported image data")

    text = pytesseract.image_to_string(image)
    return text

# Função principal para orquestrar o scraping e OCR
def main(url):
    images = scrape_images(url)
    texts = [ocr_image(img_url) for img_url in images]
    return texts

# Executando o script
url = "http://sisgou.seds.al.gov.br/base2/desaparecidos_almanaque/"
extracted_texts = main(url)
print(extracted_texts)

['ENCONTRADO\n\nInformacées: 181 Disk\n(82) 99182-0121 / 2122-5220 denunicia\nWwww.mpal.mp.br\n\nPlid/AL (ge)\n', '& i\n\nCLAUDEMIR RODRIGUES\n\nInformagées: 181 Disk\n(82) 99182-0121 / 2122-5220 denutnicia\n\nwww.mpal.mp.br\n\nQO Plid/AL\n\n', 'ENCONTRADO\n\nWwww.mpal.mp.br\n\nInformacées: 181 Disk\n(82) 99182-0121 / 2122-5220 dentnicia\n\nOriga. <> (8)\n\n', 'a\n\nJOSE APARECIDO CEZARIO DA SILVA\n\nInformagées: 181 Disk\n(82) 99182-0121 / 2122-5220 denutnicia\n\nwww.mpal.mp.br\n\nAS\n\nPlid/AL ~~ (eM)\n\n“2% Desaparecidos scuiskia econ eae ;\n', 'THIAGO RICELLI TELES DA SILVA\n\nInformagées: 181 Disk\n(82) 99182-0121 / 2122-5220 dentnicia\n\nwww.mpal.mp.br\n\nQO Plid/AL\n\n', 'ARTUR MARTILIANO DOS SANTOS FILHO\n\nInformagées: 181 Disk\n(82) 99182-0121 / 2122-5220 deniunicia\n\nwww.mpal.mp.br\n\nQO PiidaL\n\n', 'SONIA DA ROCHA LIMA\nInformagées: Disk\n(82) 99182-0121 / 2122-5220 181 dentnicia\n\nwww.mpal.mp.br\n\nPid/AL ~~ (RS)\n\n“2% Desaparecidos\n\nae\n', 'www.mpal.mp.br\n\nInforma

In [11]:
nomes = []
name_regex = r'\b[A-ZÁÉÍÓÚÂÊÎÔÛÃÕÀÇÑ]{4,}(?:[ -][A-ZÁÉÍÓÚÂÊÎÔÛÃÕÀÇÑ]{2,})*\b'
exclusions = {'ENCONTRADO', 'DESAPARECIDO', 'AL', 'QO', 'AS', 'RS', 'GFL', 'PLID', 'SO', 'IXEIRA', 'ORIG', 'DESA', 'PARECIDO', 'SWIRLS', 'SESS', 'UDLI', 'FALECIDO'}

for text in extracted_texts:
    potential_names = re.findall(name_regex, text)
    filtered_names = [name for name in potential_names if name not in exclusions]
    nomes.extend(filtered_names)

print(nomes)

['CLAUDEMIR RODRIGUES', 'JOSE APARECIDO CEZARIO DA SILVA', 'THIAGO RICELLI TELES DA SILVA', 'ARTUR MARTILIANO DOS SANTOS FILHO', 'SONIA DA ROCHA LIMA', 'ADRIANA DE ARAUJO SILVA', 'ANDRE LUIZ NUNES SILVA', 'GILBERTO DOS SANTOS SILVA', 'ATOS DE OLIVEIRA PIMENTEL', 'ANTONIO MARCOS DA SILVA RAMOS', 'IGOR SANDES DE ALBUQUERQUE', 'ARYANI VIRGINIA PAIXAO CERQUEIRA', 'IXEIRA DE MACEDO', 'VITORIA SILVESTRE DA SILVA', 'LUCAS HERCULANO PEREIRA', 'ROSANGELA LUIZA DA SILVA', 'PAULO RAFAEL NEVES REIS', 'JARDEILSON SENA AMORIM DA SILVA', 'RICHARD YURI BRITO DOS SANTOS', 'CRISTIANO ALEXANDRE DOS SANTOS', 'VALDIRENE FIRMINO DE LIMA', 'ANTONIO CARLOS DE SOUZA', 'EDSON CACHOEIRA DOS SANTOS', 'DIOGO AUGUSTO ROSENDO DA SILVA', 'JOSE BRUNO SANTOS', 'GILVAM FERREIRA DE BRITO', 'RICHARD YURI BRITO DOS SANTOS', 'JANAINA DAYANE ALAFASSOS DOS SANTOS', 'JOSIEL DIAS DE ANDRADE', 'JOSE BRUNO SANTOS', 'ADRIANO BATALHA DA SILVA', 'CARLOS EDUARDO DA SILVA VASCONCELOS', 'JOSE DENILSON ANSELMO DE SOUZA', 'TAIRYNE GABRIE

In [16]:
df = pd.DataFrame(nomes)
df.to_csv('Desaparecidos Polícia Civil AL.csv', index=False)